In [1]:
"""
This is William Franks reference sheet for Algo Trading at Northeastern
Below will include relevant functions and operations needed for the class

The information will be sorted by cells with the title at the top
"""

'\nThis is William Franks reference sheet for Algo Trading at Northeastern\nBelow will include relevant functions and operations needed for the class\n\nThe information will be sorted by cells with the title at the top\n'

In [2]:
# Imports

import numpy as np
import statistics
import requests
import csv

from scipy.stats import skew, kurtosis
from scipy import stats

import pandas as pd

from statsmodels.tsa.stattools import adfuller

In [3]:
# Mean, Variance, Skew, Kurtosis

data = [1, 5, -3, 8, 2]
data_df = pd.DataFrame()
data_df['Data'] = data

np.mean(data) # mean

statistics.variance(data) # Sample Variance
statistics.pvariance(data) # Population Variance

skew(data, bias=False) # Scipy Skew, bias=False means it IS corrected for bias
data_df['Data'].skew() # Pandas Skew

kurtosis(data, bias=False, fisher=False) # Scipy Kurtosis, fisher=True for Exkurtosis
data_df['Data'].kurtosis() # Pandas Kurtosis, defaults to Exkurtosis. 3 less than above


np.float64(-0.17474690099903256)

In [4]:
# Covariance and Correlation

x = [-1, 5, -7, 4, -10]
y = [2, -1, 14, 8, -2]
df = pd.DataFrame()
df['x'] = x
df['y'] = y

np.cov(x, y, ddof=1) # NumPy Covariance matrix (Sample Cov, ddof=1)
np.cov(x, y, ddof=1)[0, 1] # NumPy Covariance
df['x'].cov(df['y']) # Pandas Covariance (Sample Cov, n-1)
statistics.covariance(x, y) # Stats Cov

np.corrcoef(x, y) # NumPy correlation matrix
np.corrcoef(x, y)[0, 1] # NumPy correlation
df['x'].corr(df['y']) # Pandas correlation
statistics.correlation(x, y) # Stats Cor

-0.08550148560121865

In [5]:
# Hypothesis Testing
# For all the questions, different = "Statistically Different"

np.random.seed(42) # Just random data for us to use
x = np.random.normal(0.05, 0.15, 100) # mean, stddev, n
y = np.random.normal(0.02, 0.20, 100)

# Is the mean of x different from 0, SciPy 1 sample T
t_stat, p_val_1 = stats.ttest_1samp(x, 0) 

# Is the variance of x different from 0.0225 (0.15**2), SciPy 1 sample var
n = len(x)
target_var = 0.15 ** 2
chi_sq = (n - 1) * np.var(x, ddof=1) / target_var
p_val_2 = 2 * min(stats.chi2.cdf(chi_sq, n-1), stats.chi2.sf(chi_sq, n-1))

# Is mean(x) different to mean(y), SciPy 2 sample t
_, p_val_3 = stats.ttest_ind(x, y, equal_var=False) # False means Welch's T

# Is Var(x) different to Var(y), Scipy 2 sample T
_, p_val_4 = stats.levene(x, y)

# Is x normally distributed? SciPy, Jarque Bera Test
_, p_val_5 = stats.jarque_bera(x)

# Hand Calculation t stat (is mean different than 2%)
t_stat = (x.mean() - 0.02) / (x.std() / (len(x) ** 0.5)) # (xbar - null) / (stddev / (sqrt(n)))
deg_freedom = len(x) - 1
p_value = 2 * (1 - stats.t.cdf(abs(t_stat), df=deg_freedom))




In [11]:
# Alpha Vantage Processing

key='&apikey=ZKMMTO1ATDBLXH2K'
ticker='&symbol=SPY'
endpoint='function=TIME_SERIES_DAILY_ADJUSTED'
size='&outputsize=compact'
web='https://www.alphavantage.co/query?'
url =web+endpoint+ticker+size+key

r = requests.get(url)
print(r.status_code) # 200 good, 400 bad
data = r.json()

print(data.keys()) #printing the keys
time_series_data = data['Time Series (Daily)']

ts_df = pd.DataFrame.from_dict(time_series_data, orient='index').reset_index().rename(columns={'index': 'Date'})

ts_df


200
dict_keys(['Meta Data', 'Time Series (Daily)'])


,Date,1. open,2. high,3. low,4. close,5. adjusted close,6. volume,7. dividend amount,8. split coefficient
0,2026-01-13,695.49,696.09,691.35,693.77,693.77,78309726,0.0000,1.0
1,2026-01-12,690.68,696.09,690.63,695.16,695.16,63976002,0.0000,1.0
2,2026-01-09,690.63,695.31,689.18,694.07,694.07,80125467,0.0000,1.0
3,2026-01-08,688.82,690.615,687.49,689.51,689.51,64019200,0.0000,1.0
4,2026-01-07,692.19,693.96,689.32,689.58,689.58,75588337,0.0000,1.0
...,...,...,...,...,...,...,...,...,...
95,2025-08-27,644.57,647.37,644.42,646.63,642.967712971449,48341069,0.0000,1.0
96,2025-08-26,642.2,645.51,641.57,645.16,641.506038539289,51581565,0.0000,1.0
97,2025-08-25,644.04,645.29,642.35,642.47,638.83127376207,51274332,0.0000,1.0
98,2025-08-22,637.76,646.5,637.25,645.31,641.65518899155,84083214,0.0000,1.0
